In [3]:
import numpy as np

def assign_nearest_elements(A, B, m, n):
    assigned_B = []
    cost = 0

    for a in A:
        if len(B) == 0:
            cost += m
            continue

        nearest_b = min(B, key=lambda b: abs(a - b))
        assigned_B.append(nearest_b)
        B.remove(nearest_b)

    cost += n * (len(assigned_B) - 1) * min([abs(a - b) for b in assigned_B], default=0)

    return assigned_B, cost

# Example usage
A = [1.5, 2.7, 3.9, 5.1, 999]
B = [1.2, 2.4, 3.6, 4.8, 6.0]
m = 1
n = 2

assigned_B, cost = assign_nearest_elements(A, B, m, n)
print("Assigned B:", assigned_B)
print("Cost:", cost)

Assigned B: [1.2, 2.4, 3.6, 4.8, 6.0]
Cost: 7944.0


In [ ]:
www